In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fifa_df = pd.read_csv('../raw_data/completo.csv')

# 0. Separando variáveis categóricas das variáveis numéricas

In [2]:
numerical_variables = ['eur_value', 'eur_wage', 'eur_release_clause','age', 'height_cm', 'weight_kg', 'ID', 'special', 'overall', 'potential', 'pac', 'sho', 'pas', 'dri', 'def', 'phy', 'international_reputation', 'skill_moves', 'weak_foot', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength',
'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'composure', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'rs', 'rw', 'rf', 'ram', 'rcm', 'rm', 'rdm', 'rcb', 'rb', 'rwb', 'st', 'lw', 'cf', 'cam', 'cm', 'lm', 'cdm', 'cb', 'lb', 'lwb', 'ls', 'lf', 'lam', 'lcm', 'ldm', 'lcb', 'gk']
categorical_variables = [v for v in fifa_df.columns if v not in numerical_variables]

In [3]:
unuseful = ['birth_date', 'club_logo', 'photo', 'flag', 'real_face']
fifa_df.drop(unuseful, axis='columns', inplace=True)
categorical_variables = [v for v in categorical_variables if v not in unuseful]


#Categóricos não booleanos
categorical_name_variables = ['body_type', 'work_rate_att', 'work_rate_def', 'preferred_foot']

# 1. Análise Descritiva

In [4]:
missing = fifa_df.isnull().sum()
missing[missing>0]

club                    253
league                  253
eur_release_clause     1494
rs                     2021
rw                     2021
rf                     2021
ram                    2021
rcm                    2021
rm                     2021
rdm                    2021
rcb                    2021
rb                     2021
rwb                    2021
st                     2021
lw                     2021
cf                     2021
cam                    2021
cm                     2021
lm                     2021
cdm                    2021
cb                     2021
lb                     2021
lwb                    2021
ls                     2021
lf                     2021
lam                    2021
lcm                    2021
ldm                    2021
lcb                    2021
gk                    15973
dtype: int64

In [5]:
# Tirando todos os espaços em branco antes e depois das palavras
fifa_df = fifa_df.applymap(lambda x: x.strip() if type(x) is str else x)

In [6]:
fifa_df[categorical_variables].describe()

,name,full_name,club,league,body_type,real_face,nationality,work_rate_att,work_rate_def,preferred_foot,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
count,17994,17994,17741,17741,17994,17994,17994,17994,17994,17994,...,17994,17994,17994,17994,17994,17994,17994,17994,17994,17994
unique,17022,17891,647,41,9,2,164,3,3,2,...,2,2,2,1,1,1,1,1,1,2
top,J. Valencia,Scott Brown,RC Deportivo de La Coruña,Argentinian Superliga,Normal,False,England,Medium,Medium,Right,...,False,False,False,False,False,False,False,False,False,False
freq,7,3,33,780,10587,16772,1631,12447,13560,13743,...,14354,15931,17796,17994,17994,17994,17994,17994,17994,15973


In [7]:
fifa_df[categorical_name_variables].describe()

,body_type,work_rate_att,work_rate_def,preferred_foot
count,17994,17994,17994,17994
unique,9,3,3,2
top,Normal,Medium,Medium,Right
freq,10587,12447,13560,13743


In [8]:
fifa_df['work_rate_att'].value_counts()

Medium    12447
High       4645
Low         902
Name: work_rate_att, dtype: int64

In [9]:
fifa_df['work_rate_def'].value_counts()

Medium    13560
High       2860
Low        1574
Name: work_rate_def, dtype: int64

In [10]:
fifa_df['preferred_foot'].value_counts()

Right    13743
Left      4251
Name: preferred_foot, dtype: int64

#### Checando consistência das categorias de body types

In [11]:
body_types = fifa_df.body_type.value_counts()
body_types

Normal        10587
Lean           6211
Stocky         1190
Akinfenwa         1
Neymar            1
Courtois          1
Messi             1
C. Ronaldo        1
Shaqiri           1
Name: body_type, dtype: int64

In [12]:
# Recalculando body types dos jogadores com body types unicos
odd_body_types = list(body_types[body_types==1].index)
players_body = fifa_df[fifa_df['body_type'].isin(odd_body_types)]
players_body

,ID,name,full_name,club,special,age,league,height_cm,weight_kg,body_type,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,2228,32,Spanish Primera División,185.0,80.0,C. Ronaldo,...,False,False,False,False,False,False,False,False,False,False
1,158023,L. Messi,Lionel Messi,FC Barcelona,2158,30,Spanish Primera División,170.0,72.0,Messi,...,False,False,False,False,False,False,False,False,False,False
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,2100,25,French Ligue 1,175.0,68.0,Neymar,...,False,False,False,False,False,False,False,False,False,False
12,192119,T. Courtois,Thibaut Courtois,Chelsea,1282,25,English Premier League,199.0,94.0,Courtois,...,False,False,False,False,False,False,False,False,False,True
243,193348,X. Shaqiri,Xherdan Shaqiri,Stoke City,2073,25,English Premier League,169.0,72.0,Shaqiri,...,False,False,False,False,False,False,False,False,False,False
11567,156321,A. Akinfenwa,Adebayo Akinfenwa,Wycombe Wanderers,1553,35,English League Two,178.0,110.0,Akinfenwa,...,False,False,False,False,False,False,False,False,False,False


In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [14]:
regressor = AdaBoostClassifier(MultinomialNB())
scalerX = MinMaxScaler()

data_x = fifa_df.drop(players_body.index)[numerical_variables]
data_x = data_x.fillna(data_x.mean())
data_x = scalerX.fit_transform(data_x)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_y = encoder.fit_transform(fifa_df.drop(players_body.index).body_type)

results = cross_val_score(regressor, data_x, data_y, cv=10, scoring="accuracy")
np.mean(results)

0.6085183648216655

#### Acuracia:
1. knn(10):0.5527443307639636
2. knn(5): 0.5402930603723941
3. nb:
5. tree:0.4640785759731799
4. adaboost (tree): 0.5904937049785758
5. adaboos(nb): 0.6085183648216655

In [15]:
regressor = AdaBoostClassifier(MultinomialNB())
regressor.fit(data_x, data_y)
x_predict = fifa_df[numerical_variables].loc[players_body.index]
x_predict = x_predict.fillna(x_predict.mean())
predicted_body_types = regressor.predict(x_predict)

predicted_labels = encoder.inverse_transform(predicted_body_types)
predicted_labels

/home/dcandrade/.anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['Lean', 'Lean', 'Lean', 'Lean', 'Lean', 'Lean'], dtype=object)

In [16]:
fifa_df.body_type.replace({k:'Lean' for k in odd_body_types}, inplace=True)
fifa_df[fifa_df['body_type'].isin(odd_body_types)]

,ID,name,full_name,club,special,age,league,height_cm,weight_kg,body_type,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk


## 3.0 Valores Ausentes - Variáveis Categóricas 

In [17]:
fifa_df[categorical_variables].astype(str).describe()

,name,full_name,club,league,body_type,real_face,nationality,work_rate_att,work_rate_def,preferred_foot,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
count,17994,17994,17994,17994,17994,17994,17994,17994,17994,17994,...,17994,17994,17994,17994,17994,17994,17994,17994,17994,17994
unique,17022,17891,648,42,3,2,164,3,3,2,...,2,2,2,1,1,1,1,1,1,2
top,J. Valencia,Scott Brown,nan,Argentinian Superliga,Normal,False,England,Medium,Medium,Right,...,False,False,False,False,False,False,False,False,False,False
freq,7,3,253,780,10587,16772,1631,12447,13560,13743,...,14354,15931,17796,17994,17994,17994,17994,17994,17994,15973


In [18]:
# Amostra do dataset
fifa_df[categorical_variables].head()

,name,full_name,club,league,body_type,real_face,nationality,work_rate_att,work_rate_def,preferred_foot,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
0,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,Spanish Primera División,Lean,True,Portugal,High,Low,Right,...,False,False,False,False,False,False,False,False,False,False
1,L. Messi,Lionel Messi,FC Barcelona,Spanish Primera División,Lean,True,Argentina,Medium,Medium,Left,...,False,False,False,False,False,False,False,False,False,False
2,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,French Ligue 1,Lean,True,Brazil,High,Medium,Right,...,False,False,False,False,False,False,False,False,False,False
3,L. Suárez,Luis Suárez,FC Barcelona,Spanish Primera División,Normal,True,Uruguay,High,Medium,Right,...,False,False,False,False,False,False,False,False,False,False
4,M. Neuer,Manuel Neuer,FC Bayern Munich,German Bundesliga,Normal,True,Germany,Medium,Medium,Right,...,False,False,False,False,False,False,False,False,False,True


In [19]:
missing_categorical = fifa_df[categorical_variables].isnull().sum()
categorical_missing_data = missing_categorical[missing_categorical>0]
categorical_missing_data

club      253
league    253
dtype: int64

In [20]:
missing_club = fifa_df[categorical_variables][fifa_df.club.isna()].isnull().sum()
missing_club[missing_club>0]

club      253
league    253
dtype: int64

#### Todos que faltam liga também faltam o club e club_logo.  Estratégia: adotar valores padrão

In [21]:
missing_columns = list(categorical_missing_data.index)
fifa_df[missing_columns] = fifa_df[missing_columns].fillna('Desempregado')

missing_categorical = fifa_df[categorical_variables].isnull().sum()
categorical_missing_data = missing_categorical[missing_categorical>0]
categorical_missing_data

Series([], dtype: int64)

## 4. Valores Ausentes - Variáveis Numéricas

In [22]:
missing = fifa_df[numerical_variables].isnull().sum()
missing = missing[missing>0]
missing

eur_release_clause     1494
rs                     2021
rw                     2021
rf                     2021
ram                    2021
rcm                    2021
rm                     2021
rdm                    2021
rcb                    2021
rb                     2021
rwb                    2021
st                     2021
lw                     2021
cf                     2021
cam                    2021
cm                     2021
lm                     2021
cdm                    2021
cb                     2021
lb                     2021
lwb                    2021
ls                     2021
lf                     2021
lam                    2021
lcm                    2021
ldm                    2021
lcb                    2021
gk                    15973
dtype: int64

### Todos os dados ausentes, com exceção do eur_release_clause, são habilidades ausentes. Solução é igualar a zero

In [23]:
# Tratando os valores das habilidade ausentes
missing_columns = list(missing.index)
to_remove = ['eur_release_clause']
missing_columns = [c for c in missing_columns if c not in to_remove]

fifa_df[missing_columns] = fifa_df[missing_columns].fillna(0)

In [24]:
missing = fifa_df.isnull().sum()
missing = missing[missing > 0]
missing

eur_release_clause    1494
dtype: int64

#### Suposição: eur_release_clause é altamente correlacionado com o salário do jogador. Se for, remover coluna, se não for, voltar aqui e tratar os valores ausentes

In [25]:
# Tabela de correlação par a par (Coeficiente de Pearson)
fifa_df[numerical_variables].corr().head()

,eur_value,eur_wage,eur_release_clause,age,height_cm,weight_kg,ID,special,overall,potential,...,cb,lb,lwb,ls,lf,lam,lcm,ldm,lcb,gk
eur_value,1.000000,0.851631,0.994149,0.080431,0.002631,0.036923,-0.141201,0.374652,0.624772,0.584200,...,0.144834,0.172499,0.186938,0.234359,0.240734,0.241393,0.241092,0.188782,0.144834,-0.034411
eur_wage,0.851631,1.000000,0.852157,0.150949,0.020784,0.059275,-0.213648,0.366077,0.594408,0.516598,...,0.164154,0.182061,0.192563,0.223851,0.226652,0.227411,0.233268,0.197481,0.164154,-0.038325
eur_release_clause,0.994149,0.852157,1.000000,0.065595,0.001133,0.031917,-0.126660,0.365384,0.611823,0.593943,...,0.143692,0.170639,0.184308,0.228031,0.234862,0.235734,0.235730,0.186045,0.143692,-0.032771
age,0.080431,0.150949,0.065595,1.000000,0.076463,0.219722,-0.719490,0.239652,0.460902,-0.223340,...,0.109703,0.070554,0.067299,0.047406,0.032604,0.035983,0.072880,0.104208,0.109703,0.095528
height_cm,0.002631,0.020784,0.001133,0.076463,1.000000,0.766900,-0.085232,-0.380680,0.042723,0.003969,...,-0.171566,-0.307231,-0.346666,-0.391118,-0.445847,-0.456871,-0.409023,-0.277049,-0.171566,0.368455


In [26]:
def get_corrs(df):
    col_correlations = df.corr()
    col_correlations.loc[:, :] = np.tril(col_correlations, k=-1)
    cor_pairs = col_correlations.stack()
    return cor_pairs.to_dict()

correlation = get_corrs(fifa_df)

In [27]:
correlation_threshold = 0.99
correlation_pairs = {k:correlation[k] for k in correlation if correlation[k] > correlation_threshold}
correlation_pairs

{('eur_release_clause', 'eur_value'): 0.9941487501823035,
 ('rf', 'rs'): 0.9935689392656655,
 ('rf', 'rw'): 0.9973416930279289,
 ('ram', 'rw'): 0.9967966185100406,
 ('ram', 'rf'): 0.9972842992245552,
 ('rm', 'rw'): 0.9973890435899577,
 ('rm', 'rf'): 0.9938767444167369,
 ('rm', 'ram'): 0.9965187046296535,
 ('rb', 'rdm'): 0.9928553582082422,
 ('rwb', 'rb'): 0.9958621704608994,
 ('st', 'rs'): 1.0,
 ('st', 'rf'): 0.9935689392656655,
 ('lw', 'rw'): 1.0,
 ('lw', 'rf'): 0.9973416930279289,
 ('lw', 'ram'): 0.9967966185100406,
 ('lw', 'rm'): 0.9973890435899577,
 ('cf', 'rs'): 0.9935689392656655,
 ('cf', 'rw'): 0.9973416930279289,
 ('cf', 'rf'): 1.0,
 ('cf', 'ram'): 0.9972842992245552,
 ('cf', 'rm'): 0.9938767444167369,
 ('cf', 'st'): 0.9935689392656655,
 ('cf', 'lw'): 0.9973416930279289,
 ('cam', 'rw'): 0.9967966185100406,
 ('cam', 'rf'): 0.9972842992245552,
 ('cam', 'ram'): 1.0,
 ('cam', 'rm'): 0.9965187046296535,
 ('cam', 'lw'): 0.9967966185100406,
 ('cam', 'cf'): 0.9972842992245552,
 ('cm', 

In [28]:
# Identificando colunas altamente relacionadas
correlation_matrix = fifa_df[numerical_variables].corr().abs()
# Seleciona o triângulo superior da matriz de correlação
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
# Seleciona colunas a serem removidas
to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]
to_drop

['eur_release_clause',
 'rf',
 'ram',
 'rm',
 'rb',
 'rwb',
 'st',
 'lw',
 'cf',
 'cam',
 'cm',
 'lm',
 'cdm',
 'cb',
 'lb',
 'lwb',
 'ls',
 'lf',
 'lam',
 'lcm',
 'ldm',
 'lcb']

In [29]:
# Removendo entradas com altas correlação
#to_drop = 'eur_release_clause'
clean_df = fifa_df.drop(to_drop, axis='columns')
numerical_variables = [var for var in numerical_variables if var not in to_drop]

# 4.2 Normalizando os dados

In [30]:
fifa_df[numerical_variables].head()

,eur_value,eur_wage,age,height_cm,weight_kg,ID,special,overall,potential,pac,...,gk_handling,gk_kicking,gk_positioning,gk_reflexes,rs,rw,rcm,rdm,rcb,gk
0,95500000.0,565000.0,32,185.0,80.0,20801,2228,94,94,90,...,11,15,14,11,92.0,91.0,82.0,62.0,53.0,0.0
1,105000000.0,565000.0,30,170.0,72.0,158023,2158,93,93,89,...,11,15,14,8,88.0,91.0,84.0,59.0,45.0,0.0
2,123000000.0,280000.0,25,175.0,68.0,190871,2100,92,94,92,...,9,15,15,11,84.0,89.0,79.0,59.0,46.0,0.0
3,97000000.0,510000.0,30,182.0,86.0,176580,2291,92,92,82,...,25,31,33,37,88.0,87.0,80.0,65.0,58.0,0.0
4,61000000.0,230000.0,31,193.0,92.0,167495,1493,92,92,91,...,90,95,91,89,0.0,0.0,0.0,0.0,0.0,92.0


In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
clean_df[numerical_variables] = scaler.fit_transform(clean_df[numerical_variables])
clean_df[numerical_variables].head()

,eur_value,eur_wage,age,height_cm,weight_kg,ID,special,overall,potential,pac,...,gk_handling,gk_kicking,gk_positioning,gk_reflexes,rs,rw,rcm,rdm,rcb,gk
0,0.776423,1.000000,0.516129,0.60,0.508197,0.086076,0.959693,1.000000,1.000000,0.920000,...,0.111111,0.148936,0.144444,0.112360,1.000000,1.000000,0.942529,0.729412,0.609195,0.0
1,0.853659,1.000000,0.451613,0.30,0.377049,0.654346,0.914907,0.979167,0.979167,0.906667,...,0.111111,0.148936,0.144444,0.078652,0.956522,1.000000,0.965517,0.694118,0.517241,0.0
2,1.000000,0.495575,0.290323,0.40,0.311475,0.790378,0.877799,0.958333,1.000000,0.946667,...,0.088889,0.148936,0.155556,0.112360,0.913043,0.978022,0.908046,0.694118,0.528736,0.0
3,0.788618,0.902655,0.451613,0.54,0.606557,0.731196,1.000000,0.958333,0.958333,0.813333,...,0.266667,0.319149,0.355556,0.404494,0.956522,0.956044,0.919540,0.764706,0.666667,0.0
4,0.495935,0.407080,0.483871,0.76,0.704918,0.693572,0.489443,0.958333,0.958333,0.933333,...,0.988889,1.000000,1.000000,0.988764,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


# 5. Codificando colunas categóricas

In [32]:
len(fifa_df), len(fifa_df.columns)

(17994, 181)

In [33]:
## Nome e nome completo não importam no momento
to_drop = ['name', 'full_name', 'ID']
clean_df.drop(to_drop, axis='columns', inplace=True)

# Convertendo p/ binário
clean_df = pd.get_dummies(clean_df)
len(fifa_df), len(fifa_df.columns)

(17994, 181)

In [34]:
clean_df.to_csv('clean_data_normalized.csv', index=False)
fifa_df.to_csv('clean_data.csv', index=False)